In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/LLM/planty/data

/content/drive/MyDrive/LLM/planty/data


In [ ]:
!pip install openai pqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
from huggingface_hub import login
login()

## 모델 비교 코드

### 모델 사이즈

In [ ]:
import os

def get_model_size(model_name):
    # Hugging Face의 캐시 디렉토리 경로
    model_path = os.path.join(torch.hub.get_dir(), 'transformers', model_name.replace("/", "_"))

    # 모델이 캐시되었는지 확인하고 다운로드
    if not os.path.exists(model_path):
        model = AutoModelForCausalLM.from_pretrained(model_name)
        model.save_pretrained(model_path)

    # 모델 파일 크기 계산
    total_size = sum(os.path.getsize(os.path.join(model_path, f)) for f in os.listdir(model_path))
    return total_size / (1024 ** 2)  # MB 단위로 반환

## 파인튜닝 테스트 코드

### 답변 생성

In [ ]:
import os
import json
import torch
from random import sample
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

def save_answer_to_json(model_name, question, true_answer, generated_answer):
    """ 모델의 생성 결과를 JSON 파일에 저장하는 함수 """
    save_dir = "./generation_results"
    os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists
    save_path = os.path.join(save_dir, f"{model_name.replace('/', '_')}_generation_result.json")

    result_data = {
        "question": question,
        "true_answer": true_answer,
        "generated_answer": generated_answer
    }

    # 기존 데이터 불러오기 (파일이 없으면 빈 리스트 생성)
    if os.path.exists(save_path):
        with open(save_path, 'r', encoding='utf-8') as f:
            try:
                existing_data = json.load(f)
            except json.JSONDecodeError:
                existing_data = []
    else:
        existing_data = []

    # 새로운 데이터 추가
    existing_data.append(result_data)

    # JSON 파일로 저장
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)

def extract_test_data(input_file, output_file, num_samples=50):
    """ sft_dataset_cleaned.json에서 랜덤하게 50개 샘플을 추출하여 test_dataset.json으로 저장 """
    dataset = load_dataset("json", data_files=input_file, split="train")

    # 랜덤하게 50개 샘플 추출
    test_samples = sample(list(dataset), num_samples)

    # JSON 파일로 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(test_samples, f, ensure_ascii=False, indent=4)

def load_test_data(file_path):
    """ 저장된 test_dataset.json을 로드하는 함수 """
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def test_model(model_name, test_dataset):
    """ 주어진 test_dataset에 대해 모델의 답변을 생성하는 함수 """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

    for sample in tqdm(test_dataset, desc=f"Testing model {model_name}"):
        input_text = sample["question"]
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

        outputs = model.generate(
            input_ids,
            max_new_tokens=512,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.7,
            top_p=0.95
        )

        generated_answer = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

        save_answer_to_json(model_name, sample["question"], sample["answer"], generated_answer)

In [ ]:
import os
import json
import time
import torch
import csv
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

def save_answer_to_json(save_path, question, true_answer, generated_answer):
    result_data = {
        "question": question,
        "true_answer": true_answer,
        "generated_answer": generated_answer
    }

    if os.path.exists(save_path):
        with open(save_path, 'r', encoding='utf-8') as f:
            try:
                existing_data = json.load(f)
            except json.JSONDecodeError:
                existing_data = []
    else:
        existing_data = []

    existing_data.append(result_data)

    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)

def load_test_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def load_timing_data(model_name):
  csv_file_path = f"/content/drive/MyDrive/LLM/planty/data/generation_results/{model_name}/{model_name}_timing.csv"
  time = pd.read_csv(csv_file_path)
  mean_time = time['elapsed_time_sec'].mean()
  return mean_time

def test_model(model_name, test_dataset):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # 모델 로드
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

    # 경로 설정
    safe_model_name = model_name.replace('/', '_')
    save_dir = os.path.join("./generation_results", safe_model_name)
    os.makedirs(save_dir, exist_ok=True)

    json_save_path = os.path.join(save_dir, f"{safe_model_name}_generation_result.json")
    csv_save_path = os.path.join(save_dir, f"{safe_model_name}_timing.csv")

    # 추론 시간 기록용 리스트
    timing_data = []

    for idx, sample in enumerate(tqdm(test_dataset, desc=f"Testing model {model_name}")):
        input_text = sample["question"]
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

        try:
            start_time = time.time()
            outputs = model.generate(
                input_ids,
                max_new_tokens=512,
                eos_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7,
                top_p=0.95
            )
            end_time = time.time()
        except torch.cuda.OutOfMemoryError:
            torch.cuda.empty_cache()
            print(f"[OOM Error] Skipping index {idx}")
            continue

        elapsed_time = end_time - start_time

        generated_answer = tokenizer.decode(
            outputs[0][input_ids.shape[-1]:], skip_special_tokens=True
        )

        save_answer_to_json(json_save_path, sample["question"], sample["answer"], generated_answer)

        timing_data.append({
            "index": idx,
            "question": sample["question"],
            "elapsed_time_sec": round(elapsed_time, 4)
        })

    with open(csv_save_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["index", "question", "elapsed_time_sec"])
        writer.writeheader()
        writer.writerows(timing_data)

    print(f"JSON 저장 완료: {json_save_path}")
    print(f"추론 시간 CSV 저장 완료: {csv_save_path}")

In [ ]:
output_file = "./sft_dataset/sft_test.json"

### 답변 평가

In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
from functools import partial
from pqdm.processes import pqdm
from openai import OpenAI
from pydantic import BaseModel

def get_openai_client():
    return OpenAI(api_key="your_openai_key")

class Criterion(BaseModel):
    score: int
    explanation: str

class PlantEvaluation(BaseModel):
    plant_info_accuracy: Criterion
    answer_appropriateness: Criterion

def evaluate_qa_pair(idx, qa_pairs, model_name):
    client = get_openai_client()
    folder_name = model_name.replace('/', '_')
    save_dir = f"./qa_evaluation_results/{folder_name}"
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"result_{idx}.json")

    if Path(save_path).exists():
        print(f"인덱스 {idx}에 대한 결과가 이미 존재합니다. ({model_name})")
        return None

    question = qa_pairs[idx]["question"]
    reference_answer = qa_pairs[idx]["true_answer"]
    model_answer = qa_pairs[idx]["generated_answer"]

    prompt = f"""
질문: {question}
참조 답변: {reference_answer}
모델 생성 답변: {model_answer}

위의 질문과 답변을 평가해주세요. 특히 답변이 식물에 대한 것인지 여부를 고려하여 다음 두 가지 기준에 따라 1-10점 사이의 점수를 매겨주세요:

1. 식물 정보 정확성: 모델의 답변에 포함된 식물 관련 정보가 참조 답변과 비교하여 얼마나 정확한가? (만약 질문이 식물에 관한 것이 아니라면 이 항목은 건너뛰고 설명을 "해당 없음"으로 해주세요.)
2. 답변 적절성: 모델의 답변이 질문에 얼마나 적절하고 유용한가?

각 기준에 대한 점수와 간단한 설명을 제공해주세요.
"""

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "식물 관련 QA 모델의 응답을 평가하는 임무를 맡은 AI 어시스턴트입니다."},
                {"role": "user", "content": prompt}
            ]
        )

        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(completion.model_dump(), f, ensure_ascii=False, indent=4)

        return completion

    except Exception as e:
        print(f"Error processing index {idx} ({model_name}): {str(e)}")
        return None

def model_evaluate(model_name):
    input_file = f"./generation_results/{model_name}/{model_name}_generation_result.json"

    if not Path(input_file).exists():
        print(f"파일을 찾을 수 없습니다: {input_file}")
        return

    with open(input_file, "r", encoding="utf-8") as f:
        try:
            qa_pairs = json.load(f) # JSON 파일에서 리스트로 데이터 로드

        except json.JSONDecodeError as e:
            print(f"JSON 디코딩 오류 발생: {str(e)}")
            return

    indexed_qa_pairs = list(range(len(qa_pairs)))
    evaluate_func = partial(evaluate_qa_pair, qa_pairs=qa_pairs, model_name=model_name)
    results = pqdm(indexed_qa_pairs, evaluate_func, n_jobs=40)

### 채점 점수

In [ ]:
import json
from glob import glob
import re

def view_score(model_name):
    path = f"./qa_evaluation_results/{model_name}"
    path_list = glob(f"{path}/*.json")

    plant_scores = []
    answer_scores = []
    processed_files = 0

    for path in path_list:
        with open(path, "r") as file:
            try:
                data = json.load(file)
                if "choices" in data and len(data["choices"]) > 0 and "message" in data["choices"][0] and "content" in data["choices"][0]["message"]:
                    content = data["choices"][0]["message"]["content"]

                    # Extract scores using regex
                    plant_match = re.search(r"식물 정보 정확성: (\d+)(?:/10)?점", content)
                    answer_match = re.search(r"답변 적절성: (\d+)(?:/10)?점", content)

                    if plant_match:
                        plant_scores.append(int(plant_match.group(1)))
                    if answer_match:
                        answer_scores.append(int(answer_match.group(1)))

                    processed_files += 1
                else:
                    print(f"Skipping file {path} due to unexpected structure.")
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in {path}: {e}")
            except Exception as e:
                print(f"Error processing file {path}: {e}")

    if processed_files == 0:
        print("점수가 포함된 파일이 없습니다.")
        return

    if plant_scores:
        mean_plant_score = sum(plant_scores) / len(plant_scores)
        print(f"평균 식물 정보 정확성 점수: {mean_plant_score}")
    else:
        print("식물 정보 정확성 점수가 없습니다.")

    if answer_scores:
        mean_answer_score = sum(answer_scores) / len(answer_scores)
        print(f"평균 답변 적절성 점수: {mean_answer_score}")
    else:
        print("답변 적절성 점수가 없습니다.")


## google/gemma-3-4b-it

In [ ]:
# 저장된 test_dataset.json 불러오기
test_data = load_test_data(output_file)

# gemma-3-4b 모델 테스트
test_model("google/gemma-3-4b-it", test_data)

NameError: name 'output_file' is not defined

In [ ]:
model_evaluate("google_gemma-3-4b-it")

QUEUEING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
view_score("google_gemma-3-4b-it")

평균 식물 정보 정확성 점수: 8.594594594594595
평균 답변 적절성 점수: 8.76923076923077


In [ ]:
# 모델 사이즈 구하기
model_size = get_model_size("google/gemma-3-4b-it")
print(f"모델 크기: {model_size:.2f} MB")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

모델 크기: 16403.70 MB


In [ ]:
# 모델 답변 시간
load_timing_data("google_gemma-3-4b-it")

np.float64(36.252798000000006)

## yerim00/gemma_sft

In [ ]:
# 저장된 test_dataset.json 불러오기
test_data = load_test_data(output_file)

# 파인튜닝한 gemma 모델 테스트
test_model("yerim00/gemma_sft", test_data)

Using device: cuda


adapter_config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/8.93M [00:00<?, ?B/s]

Loading adapter weights from yerim00/gemma_sft led to unexpected keys not found in the model: model.layers.0.self_attn.q_proj.lora_A.default.weight, model.layers.0.self_attn.q_proj.lora_B.default.weight, model.layers.0.self_attn.v_proj.lora_A.default.weight, model.layers.0.self_attn.v_proj.lora_B.default.weight, model.layers.1.self_attn.q_proj.lora_A.default.weight, model.layers.1.self_attn.q_proj.lora_B.default.weight, model.layers.1.self_attn.v_proj.lora_A.default.weight, model.layers.1.self_attn.v_proj.lora_B.default.weight, model.layers.10.self_attn.q_proj.lora_A.default.weight, model.layers.10.self_attn.q_proj.lora_B.default.weight, model.layers.10.self_attn.v_proj.lora_A.default.weight, model.layers.10.self_attn.v_proj.lora_B.default.weight, model.layers.11.self_attn.q_proj.lora_A.default.weight, model.layers.11.self_attn.q_proj.lora_B.default.weight, model.layers.11.self_attn.v_proj.lora_A.default.weight, model.layers.11.self_attn.v_proj.lora_B.default.weight, model.layers.12.se

...

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Testing model yerim00/gemma_sft:   0%|          | 0/50 [00:00<?, ?it/s]

JSON 저장 완료: ./generation_results/yerim00_gemma_sft/yerim00_gemma_sft_generation_result.json
추론 시간 CSV 저장 완료: ./generation_results/yerim00_gemma_sft/yerim00_gemma_sft_timing.csv


In [ ]:
model_evaluate("yerim00_gemma_sft")

QUEUEING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
view_score("yerim00_gemma_sft")

평균 식물 정보 정확성 점수: 8.88888888888889
평균 답변 적절성 점수: 8.58974358974359


In [ ]:
# 모델 사이즈 구하기
model_size = get_model_size("yerim00/gemma_sft")
print(f"모델 크기: {model_size:.2f} MB")

모델 크기: 12.33 MB


In [ ]:
# 모델 답변 시간
load_timing_data("yerim00_gemma_sft")

np.float64(42.073834)

## yerim00/gemma-3-4b-it-planty-2

In [ ]:
# 저장된 test_dataset.json 불러오기
test_data = load_test_data(output_file)

# gemma-3-4b 모델 테스트
test_model("yerim00/gemma-3-4b-it-planty-2", test_data)

Using device: cuda


adapter_config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Testing model yerim00/gemma-3-4b-it-planty-2:   0%|          | 0/50 [00:00<?, ?it/s]

JSON 저장 완료: ./generation_results/yerim00_gemma-3-4b-it-planty-2/yerim00_gemma-3-4b-it-planty-2_generation_result.json
추론 시간 CSV 저장 완료: ./generation_results/yerim00_gemma-3-4b-it-planty-2/yerim00_gemma-3-4b-it-planty-2_timing.csv


In [ ]:
model_evaluate("yerim00_gemma-3-4b-it-planty-2")

QUEUEING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
view_score("yerim00_gemma-3-4b-it-planty-2")

평균 식물 정보 정확성 점수: 7.5
평균 답변 적절성 점수: 7.242424242424242


In [ ]:
# 모델 사이즈 구하기
model_size = get_model_size("yerim00/gemma-3-4b-it-planty-2")
print(f"모델 크기: {model_size:.2f} MB")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

모델 크기: 45.46 MB


In [ ]:
# 모델 답변 시간
load_timing_data("yerim00_gemma-3-4b-it-planty-2")

np.float64(47.165152)

## yerim00/gemma-3-4b-it-planty-ia3

In [ ]:
# 저장된 test_dataset.json 불러오기
test_data = load_test_data(output_file)

# gemma-3-4b 모델 테스트
test_model("yerim00/gemma-3-4b-it-planty-ia3", test_data)

Using device: cuda


adapter_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Testing model yerim00/gemma-3-4b-it-planty-ia3:   0%|          | 0/50 [00:00<?, ?it/s]

JSON 저장 완료: ./generation_results/yerim00_gemma-3-4b-it-planty-ia3/yerim00_gemma-3-4b-it-planty-ia3_generation_result.json
추론 시간 CSV 저장 완료: ./generation_results/yerim00_gemma-3-4b-it-planty-ia3/yerim00_gemma-3-4b-it-planty-ia3_timing.csv


In [ ]:
model_evaluate("yerim00_gemma-3-4b-it-planty-ia3")

QUEUEING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
view_score("yerim00_gemma-3-4b-it-planty-ia3")

평균 식물 정보 정확성 점수: 9.121212121212121
평균 답변 적절성 점수: 8.538461538461538


In [ ]:
# 모델 사이즈 구하기
model_size = get_model_size("yerim00/gemma-3-4b-it-planty-ia3")
print(f"모델 크기: {model_size:.2f} MB")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

모델 크기: 1.65 MB


In [ ]:
# 모델 답변 시간
load_timing_data("yerim00_gemma-3-4b-it-planty-ia3")

np.float64(41.268195999999996)

## naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B

In [ ]:
# 저장된 test_dataset.json 불러오기
test_data = load_test_data(output_file)

# gemma-3-4b 모델 테스트
test_model("naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B", test_data)

Using device: cuda


config.json:   0%|          | 0.00/764 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.17G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Testing model naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B:   0%|          | 0/50 [00:00<?, ?it/s]

JSON 저장 완료: ./generation_results/naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B/naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B_generation_result.json
추론 시간 CSV 저장 완료: ./generation_results/naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B/naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B_timing.csv


In [ ]:
model_evaluate("naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B")

QUEUEING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
view_score("naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B")

평균 식물 정보 정확성 점수: 6.6
평균 답변 적절성 점수: 6.294117647058823


In [ ]:
# 모델 사이즈 구하기
model_size = get_model_size("naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B")
print(f"모델 크기: {model_size:.2f} MB")

모델 크기: 6048.42 MB


In [ ]:
# 모델 답변 시간
load_timing_data("naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B")

np.float64(7.075184)

## yerim00/HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3

In [ ]:
# 저장된 test_dataset.json 불러오기
test_data = load_test_data(output_file)

# gemma-3-4b 모델 테스트
test_model("yerim00/HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3", test_data)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/764 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.17G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/8.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Testing model yerim00/HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3:   0%|          | 0/50 [00:00<?, ?it/s]

JSON 저장 완료: ./generation_results/yerim00_HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3/yerim00_HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3_generation_result.json
추론 시간 CSV 저장 완료: ./generation_results/yerim00_HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3/yerim00_HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3_timing.csv


In [ ]:
model_evaluate("yerim00_HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3")

QUEUEING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
view_score("yerim00_HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3")

평균 식물 정보 정확성 점수: 6.310344827586207
평균 답변 적절성 점수: 6.172413793103448


In [ ]:
# 모델 사이즈 구하기
model_size = get_model_size("yerim00/HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3")
print(f"모델 크기: {model_size:.2f} MB")

모델 크기: 0.76 MB


In [ ]:
# 모델 답변 시간
load_timing_data("yerim00_HyperCLOVAX-SEED-Text-Instruct-1.5B-planty-ia3")

np.float64(8.804778)

## naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B

In [ ]:
# 저장된 test_dataset.json 불러오기
test_data = load_test_data(output_file)

# gemma-3-4b 모델 테스트
test_model("naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B", test_data)

Using device: cuda


config.json:   0%|          | 0.00/764 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.17G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Testing model naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B:   0%|          | 0/50 [00:00<?, ?it/s]

JSON 저장 완료: ./generation_results/naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B/naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B_generation_result.json
추론 시간 CSV 저장 완료: ./generation_results/naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B/naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B_timing.csv


In [ ]:
model_evaluate("naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B")

QUEUEING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/50 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
view_score("naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B")

평균 식물 정보 정확성 점수: 6.6
평균 답변 적절성 점수: 6.294117647058823


In [ ]:
# 모델 사이즈 구하기
model_size = get_model_size("naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B")
print(f"모델 크기: {model_size:.2f} MB")

모델 크기: 6048.42 MB


In [ ]:
# 모델 답변 시간
load_timing_data("naver-hyperclovax_HyperCLOVAX-SEED-Text-Instruct-1.5B")

np.float64(7.075184)